# GARCH(1, 1) model

Following the slides, given that $r_t = \mu + \sigma_t \epsilon_t$,

$$\sigma_t^2 = \alpha_0 + \alpha_1 a_{t-1}^2 + \beta_1 \sigma_{t-1}^2,$$
    
where $a_t = \sigma_t \epsilon_t$.

(a) Estimate the parameters of a GARCH(1,1) model

Given a parameter set $\mathbf{\theta} = \{\theta, \sigma_0, \mu, \alpha_0, \alpha_1, \beta_1\}$ and a sequence of stock return $\{r_t\}$, we can first calculate $\{a_t\}$.

And based on $\sigma_t^2 = \alpha_0 + \alpha_1 a_{t-1}^2 + \beta_1 \sigma_{t-1}^2$ with our parameter set, we can calculate $\{ \sigma_t \}$.

Then, from class we have, the log-likelihood of GARCH(1,1) as,
$$
\ln \mathcal{L}(\sigma_0, \alpha_0, \alpha_1, \beta_1, \mu \mid r_1, r_2, \dots, r_T) = -\frac{T}{2} \ln(2 \pi) - \frac{1}{2} \sum_{i=1}^T \ln \sigma_i^2 - \frac{1}{2} \sum_{i=1}^T \left(\frac{(r_i - \mu)^2}{\sigma_i^2}\right)
$$

Download SPY and calculate daily return and store them into a pickle file.

In [87]:
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta

In [88]:
spy_data = yf.download("SPY", start="2015-01-02", end="2024-10-26")
spy_data['Daily Return'] = spy_data['Adj Close'].pct_change()
spy_data.dropna(inplace=True)

with open("spy_daily_returns.pkl", "wb") as f:
    pickle.dump(spy_data['Daily Return'], f)

print("Data saved to spy_daily_returns.pkl")

[*********************100%%**********************]  1 of 1 completed
Data saved to spy_daily_returns.pkl


In [89]:
# Load the pickle file
with open("spy_daily_returns.pkl", "rb") as f:
    spy_daily_returns = pickle.load(f)

# Convert to DataFrame if it's not already one
if not isinstance(spy_daily_returns, pd.DataFrame):
    spy_daily_returns = pd.DataFrame(spy_daily_returns)

spy_daily_returns.head()

,Daily Return
Date,
2015-01-05,-0.018060
2015-01-06,-0.009419
2015-01-07,0.012461
2015-01-08,0.017745
2015-01-09,-0.008014


In [91]:
r = spy_daily_returns.values
time_index = spy_daily_returns.index

In [92]:
def neg_likelihood_func(r, mu, sigma):
    var = sigma ** 2
    val = (np.sum((r - mu) ** 2 / var) + 0.5 * np.log(
        var).sum())  # Dropping out constant term since it doesnt matter to have that
    return val

In [93]:
from scipy.optimize import minimize

def neg_likelihood_func_with_params(params):
    alpha0, alpha1, beta1, mu, sigma0 = params
    N = len(r)
    sigma = np.zeros((N, 1))
    sigma[0] = sigma0
    a = r - mu
    squared_a = a ** 2
    for i in range(1, N):
        sigma[i] = alpha0 + alpha1 * squared_a[i - 1] + beta1 * sigma[i - 1]
    neg_log_val = neg_likelihood_func(r, mu, sigma)
    return neg_log_val


x0 = [np.std(r), 0.1, 0.1, np.mean(r), np.std(r)]
result = minimize(neg_likelihood_func_with_params, x0, method='SLSQP', tol=1e-6)

print("Optimized Parameters:", result.x)
print("Minimum Negative Log-Likelihood:", result.fun)

/var/folders/tk/r1y674y91sg61lq0_q0y326h0000gn/T/ipykernel_71729/3305024071.py:11: RuntimeWarning: overflow encountered in multiply
  sigma[i] = alpha0 + alpha1 * squared_a[i - 1] + beta1 * sigma[i - 1]
/var/folders/tk/r1y674y91sg61lq0_q0y326h0000gn/T/ipykernel_71729/2958338494.py:2: RuntimeWarning: overflow encountered in square
  var = sigma ** 2


Optimized Parameters: [ 3.40509101e-02  5.32370226e+00 -9.19589587e-01  3.78026481e-03
  1.01424409e-01]
Minimum Negative Log-Likelihood: -8694.085290640227
